<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv(url)

In [3]:
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** $a$ не изменятся

**Обоснование:** 

A - обратимая матрица

E - Единичная матрица

Умножим признаки на обратимую матрицу:
$$ 
a = XA((XA)^T XA)^{-1} (XA)^T y = XAA^{-1}(X^T X)^{-1} (A^T)^{-1} A^T X^T y = XE(X^T X)^{-1} E X^T y = X(X^T X)^{-1} X^T y = Xw 
$$

## Алгоритм преобразования

**Алгоритм**

Создадать случайную, обратимую матрицу A и умножить признаки матрицы X на матрицу A

**Обоснование**

$$ 
a = Xw = XAw_A
$$

## Проверка алгоритма

In [5]:
X = data.drop(columns='Страховые выплаты')
y = data['Страховые выплаты']

In [6]:
class transformations:
    def fit(self, X):
        self.A = np.random.normal(size=(X.shape[1], X.shape[1]))
        self.X = X
        return self
    
    def transform(self):  
        return self.X.dot(self.A)
    
    def inv(self):
        return np.linalg.inv(self.A)

In [7]:
X_new = transformations().fit(X)
X_inv = X_new.inv()
X_new = X_new.transform()
X_inv = X_new.dot(X_inv)

In [8]:
lin = LinearRegression()

lin.fit(X, y)
predict1 = lin.predict(X)

lin.fit(X_new, y)
predict2 = lin.predict(X_new)

lin.fit(X_inv, y)
predict3 = lin.predict(X_inv)

print(np.isclose(r2_score(y, predict1), r2_score(y, predict2)))
(r2_score(y, predict1), r2_score(y, predict2), r2_score(y, predict3))

True


(0.42494550286668, 0.4249455028666794, 0.424945502866679)

### Вывод

Качество линейной регрессии из sklearn существенно не отличается до и после преобразования.